In [2]:
import pandas as pd
import numpy as np

In [3]:
data=pd.read_csv('data-ready-sample.csv')

C:\Users\Nicholas\AppData\Local\Programs\Python\Python36\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


def concat(year):
    #filename='Data_'+str(year)+'_ready.csv'
    #Local
    filename='d'+year+'.csv'
    subset=pd.read_csv(filename)
    subset=subset.drop(['temp_flag'], axis=1)
    global data
    data=pd.concat([data,subset], axis=0)

years=['00', '01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13'] 
data=pd.DataFrame() 
for year in years: 
    concat(year)

In [6]:
#data.to_csv('data-ready-sample.csv', index= False)

In [11]:
#Local
#data=data.sample(frac=0.2)

In [4]:
try:
    data=data[data['month']!='month']
    data=data[data['day']!='day']
    data=data[data['hour']!='hour']
except BaseException:
    pass

In [5]:
data=data.dropna()
#data=data.drop(['temp_flag'], axis=1)
data['month']=data['month'].apply(lambda x: str(x))
data['hour']=data['hour'].apply(lambda x: str(x))
data['day']=data['day'].apply(lambda x: str(x))
data['year']=data['year'].apply(lambda x: str(x))
pre_dummies=pd.concat([data['hour'], data['day'], data['month'], data['year']], axis=1)
dummies=pd.get_dummies(pre_dummies)
data=pd.concat([data, dummies], axis=1)
data=data.drop_duplicates()
data=data.drop(['hour', 'day', 'month', 'year'], axis=1)

In [6]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
data = scaler.fit_transform(data)

In [59]:
data.shape

(1432753, 94)

In [7]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
    #for i in range(1):
        a = dataset[i:(i+look_back), :]
        dataX.append(a)
        dataY.append(data[i+look_back,8])
    dataX=np.delete(dataX, 8, axis=2)
    return np.array(dataX), np.array(dataY)

In [48]:
#y = np.array(data[:, 8])
#X=np.delete(data, 8, axis=1)

In [8]:
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense, Activation, Flatten, Dropout
from keras.callbacks import ModelCheckpoint

C:\Users\Nicholas\AppData\Local\Programs\Python\Python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [9]:
# checkpoint
filepath="checkpoint-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='mae', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [10]:
#Local
lb=6
X, y=create_dataset(data, lb)

In [14]:
# Using Skicit-learn to split data into training and testing sets
from sklearn.model_selection import train_test_split
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 1)

In [15]:
# create and fit the LSTM network
model = Sequential()
model.add(LSTM(6, input_dim=93))
model.add(Dense(1))
#Using default sigmoid activation function
model.compile(loss='mae', optimizer='adam', metrics=['mae'])
model.fit(X_train, y_train, epochs=1, batch_size=1, verbose=1)

C:\Users\Nicholas\AppData\Local\Programs\Python\Python36\lib\site-packages\ipykernel_launcher.py:3: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Nicholas\AppData\Local\Programs\Python\Python36\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(6, input_shape=(None, 93))`
  This is separate from the ipykernel package so we can avoid doing imports until


Epoch 1/1
1074559/1074559 [==============================] - 4627s 4ms/step - loss: 0.0190 - mean_absolute_error: 0.0190


In [17]:
model.save('sample_o3_lstm.h5')